# Item 16: Prefer `get` over `in` and `KeyError` to Handle Missing Dictionary Keys

The contents of dictionaries are dynamic, and thus it's entirely possible-even likely-that when we try to access or delete a key, it won't already be present.

In [2]:
counters = {
    'pumpernickel': 2,
    'sourdough': 1
}

To increment the counter, we need to see oif the key exists, insert the key with a default counter value of 0 if it's missing, and the increment the counter's value. This requires accessing the key two times and assigning it once. 

In [3]:
# Here we use an if statement to accomplish this which returns True if the key exists
key = 'wheat'

if key in counters:
    count = counters[key]
else:
    count = 0

counters[key] = count + 1

In [4]:
# Here we accomplish the same behavior as the code above, but this time by relyinh on how dictionaries raise a
# KeyError exception whe we try to get the value for a key that doesn't exist. This approach is more efficient
# because it requires only one access and one assignment
try:
    count = counters[key]
except KeyError:
    count = 0

counters[key] = count + 1

This flow of fetching a key that exists or returning a default value is so common that the `dict` built-in type provides the `get` method to accomplish this task. The second parameter to `get` is the default value to return in case that the key-the first parameter-isn't present. 

In [7]:
# The approach mentioned above also requires only one access and one assignment, but it's much shorter than the
# KeyError example. This approach is the shortest and clearest option.
count = counters.get(key, 0)
counters[key] = count + 1

In [8]:
# We can shorten the in and KeyError approaches, but these alternatives will suffer from requiring code
# duplication for the assignments, which makes them less readable and worth avoiding
if key not in counters:
    counters[key] = 0
counters[key] += 1

if key in counters:
    counters[key] += 1
else:
    counters[key] = 1

# -----------------
try:
    counters[key] += 1
except KeyError:
    counters[key] = 1


In [15]:
# What if the values of the dictionary are a more complex type, like a list?
# Relying on the in expression requires two accesses if the key is present, or one access and one assignment
# if the key is missing
votes = {
    'baguette': ['Bob', 'Alice'],
    'ciabtta': ['Coco', 'Deb']
}

key = 'brioche'
who = 'Elmer'

if key in votes:
    names = votes[key]
else:
    votes[key] = names = []

names.append(who)
print(votes)

{'baguette': ['Bob', 'Alice'], 'ciabtta': ['Coco', 'Deb'], 'brioche': ['Elmer']}


In [16]:
# We can also rely on the KeyError exception being raised when the dictionary value is a list. This approach
# requires one key access if the key is present, or one key accessa and one assignment if it's missing, which
# makes it more efficient than the in condition
try: 
    names = votes[key]
except KeyError:
    votes[key] = names = []

names.append(who)

In [17]:
# We can also use the get method to fetch list values when the key is present, or do one fetch and one assignment
# if the key isn't present
names = votes.get(key)
if names:
    votes[key] = names = []

In [18]:
if names := votes.get(key):
    votes[key] = names = []

The `dict` type also provides the `setdefault` method to help shorten this pattern even further. `setdefault` tries to fetch the value of a key in the dictionary. If the key isn't present, the method assigns that key to the default value provided. And then the method returns the value for that key

In [19]:
# This works as expected, and it is shorter than using get with an assignment expression but it is not as
# readable
names = votes.setdefault(key, [])
names.append(who)

In [22]:
# Important: the default value passed to setdefault is assigned directly into the dictionary when the key is
# missing instead of being copied
data = {}
key = 'foo'
value = []
data.setdefault(key, value)
print('Before: ', data)
value.append('hello')
print('After: ', data)

Before:  {'foo': []}
After:  {'foo': ['hello']}


The above gotcha means that we need to make sure that we're always constructing a new default value for each key we access with `setdefault`. This leads to significant performance overhead in this example because we have to allocate a `list` instance for each call. If we resuse an object for the default value-which we might try to do to increase efficiency or readability-we might introduce strange behavior and bugs into our code.

In [23]:
# Lets implement setdefault to our earlier example
count = counters.setdefault(key, 0)
counters[key] = count + 1